In [1]:
import random as rd
import numpy as np

from glob import glob

In [2]:
def euclid_extended(a, b):
    '''
    Realiza o cálculo do mdc
    entre a e b, retornando
    o mdc e x e y tais que
    ax + by = mdc(a, b).
    '''
    inverted = False
    if b > a:
        a, b = b, a
        inverted = True
        
    
    table = np.array([[a, b], [1, 0], [0, 1]])
    iteration = 0
    while table[0, (iteration + 1) % 2] != 0:
        a, b = table[0, iteration % 2], table[0, (iteration + 1) % 2]
        q = a // b
        r = a % b
        table[:, iteration % 2] -= table[:, (iteration + 1) % 2] * q
        iteration += 1
        
    lcd, x, y = table[:, iteration % 2]
    
    if inverted:
        return lcd, y, x
    else:
        return lcd, x, y

def is_prime(n):
    '''
    Recebe um número e verifica
    se ele é primo
    '''
    if n == 2:
        return True
    
    if n < 2 or n % 2 == 0:
        return False
    
    for i in range(3, int(n**(0.5)) + 1, 2):
        if n % i == 0:
            return False
        
    return True

In [3]:
# Buscando os primos de 10.000 até
# 19.999

if 'primos.npy' not in glob('*.npy'):
    primes = []
    a = 10000
    for i in range(5000):
        if is_prime(a + 2 * i + 1):
            primes.append(a + 2 * i + 1)

    primes = np.array(primes)
    with open('primos.npy', 'wb') as f:
        np.save(f, primos)

else:
    primes = np.load('primos.npy')

primes

array([10007, 10009, 10037, ..., 19991, 19993, 19997])

In [4]:
def choose_parameters(primes):
    '''
    Recebe um array de primos e
    retorna dois primos p e q,
    phi(n) e números e e d de
    modo que de = 1 (mod phi(n)),
    onde n = p*q
    '''
    
    rd.shuffle(primes)
    p, q = primes[:2]
    phi_n = (p - 1) * (q - 1)
    
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
    
    return p, q, phi_n, e, d

def str2int(message):
    exp = 1
    number = 0
    message = message[::-1]
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    message = message[::-1]
    return message

def generate_keys(p, q):
    phi_n = (p - 1) * (q - 1)
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
        
    return (phi_n, e), (phi_n, d)

def encrypt(message, public_key):
    phi_n, e = public_key
    phi_n = int(phi_n)
    m = str2int(message)
    m = (m * e) % phi_n
    encrypted = int2str(m)
    
    return encrypted

def decrypt(encrypted, private_key):
    phi_n, d = private_key
    d = int(d)
    phi_n = int(phi_n)
    m = str2int(encrypted)
    m = (m * d) % phi_n
    message = int2str(m)
    
    return message

In [5]:
# alguns primos de https://en.wikipedia.org/wiki/Largest_known_prime_number
primes = [999999000001, 67280421310721, 170141183460469231731687303715884105727, 20988936657440586486151264256610222593863921, 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127, 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151, 10407932194664399081925240327364085538615262247266704805319112350403608059673360298012239441732324184842421613954281007791383566248323464908139906605677320762924129509389220345773183349661583550472959420547689811211693677147548478866962501384438260291732348885311160828538416585028255604666224831890918801847068222203140521026698435488732958028878050869736186900714720710555703168729087]

In [6]:
message = 'O eclass ta fora do ar tbm? sei q zap, face e insta tão fora, mas tbm não consigo entrar no eclass. Como q vou estudar algebra? como q vou fazer a lista pra amanha?'
# p, q = 100000000000000003, 100000000000000013
p, q = primes[-1], primes[-2]

public_key, private_key = generate_keys(p, q)
e_message = encrypt(message, public_key)
d_message = decrypt(e_message, private_key)
print(e_message)
print(d_message)

²ïYVÔ0}cÌhRç gÚ1~ 9»"3BFÓx8sÿ½³1Dgé9Þ Êó ?ÝVÓ9µî,-£
O eclass ta fora do ar tbm? sei q zap, face e insta tão fora, mas tbm não consigo entrar no eclass. Como q vou estudar algebra? como q vou fazer a lista pra amanha?


In [7]:
import smtplib
from getpass import getpass
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_mail(from_mail, to_mail, subject, body, public_key):
    password = getpass(prompt = 'Password: ')
    body = encrypt(body, public_key)
    msg = MIMEMultipart()
    msg['From'] = from_mail
    msg['To'] = to_mail
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(from_mail, password)
    text = msg.as_string()
    server.sendmail(from_mail, to_mail, text)
    server.quit()
    
    return body

In [8]:
from_mail = 'igorpmichels@gmail.com'
to_mail = 'luca.escopelli@gmail.com'
# to_mail = 'igorpmichels@gmail.com'
subject = 'Hey man'
body = message
send_mail(from_mail, to_mail, subject, body, public_key)

Password: ········


'Ü¦\\Ý\x9f\x91U\x0eE\x88F½×9>åµsµ*ý$©±EÎ×¯ïÓ¯+Y¹×èÚÃ³S\x8c\x1fJ\x0b\x8eô§Õ\x11F\x15T#\xa0,\x88iË\x9e\x99\x96ñLÉð¤\x0fM\x16I7\x10%A\x0fFtØ\x1bã\x84Î\x1c\x98Åe\x17BVÜ\x1eÁxù[À\x1e\x12³n:GeI\x91º\'\x82E»ùù\x81.\x07z\x97\x7fâh\x85(\xa0Qó\x91\x86¿ÐÃN.»oýæF ÊÐ\x00\x8fèÂ\x92\x17;è¿\x98\x84\r\x95\x89²ïYV\x92Ô0}cÌ\x89hRç g\x17Ú1\x01~\x009»\x99"\x063\x81BFÓx\x87\x0c\x178s\x9bÿ½³\x961Dg\x1bé\x079Þ\x0b\x00Êó\x95\x00\x91?ÝVÓ9µî,\x9f\x9f\x92-£'

In [9]:
import smtplib
import time
import imaplib
import email
import traceback 

def read_email_from_gmail(username, password, server, addres, private_key, qtd = 1):
    printed = 0
    try:
        mail = imaplib.IMAP4_SSL(server)
        mail.login(username, password)
        mail.select('inbox')
        data = mail.search(None, 'ALL')
        mail_ids = data[1]
        id_list = mail_ids[0].split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])
        for i in range(latest_email_id, first_email_id, -1):
            if printed == qtd:
                break
                
            data = mail.fetch(str(i), '(RFC822)')
            for response_part in data:
                arr = response_part[0]
                if isinstance(arr, tuple):
                    msg = email.message_from_string(str(arr[1], 'utf-8'))
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print(type(email_from), email_from)
                    if email_from != addres:
                        break
                    else:
                        printed += 1
                    
                    print('From : ' + email_from + '\n')
                    print('Subject : ' + email_subject + '\n')
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = str(part.get('Content-Disposition'))
                        try:
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass
                        if content_type == 'text/plain':
                            print(body)
                            body = decrypt(body, private_key)
                            print(body)

    except Exception as e:
        traceback.print_exc() 
        print(str(e))

username = 'igorpmichels@gmail.com'
password = getpass(prompt = 'Password: ')
server = 'imap.gmail.com'
# addres = 'luca.escopelli@gmail.com'
addres = 'igorpmichels@gmail.com'
read_email_from_gmail(username, password, server, addres, private_key)

Password: ········
<class 'str'> igorpmichels@gmail.com
From : igorpmichels@gmail.com

Subject : Hey man

\L)tå3ò¶Å¥aÔE¢1b×Ìcl1¹®É_3Ç­0Xl´)Ëüà@{èÎÕû4@®òc±²jðét©ßºrëuÚ­8bjí¶¬ýíCwËe	|&sçaHtz¹ÑG:L&ï&Iã>Îç2kðÝn÷Ò ¶¢7«¬Xe§ØÑhè7ÀVKÝ|2ï(Y?ÙC84wW¡×sN;q¶Û$ùçvLA¡]Õÿ%4`,i¼k~Û6ïÕÖC+ÿ sý
Òc(Î¾!L[Rïusm²Fß8®ÕZèåÈvFPën JôØÒè6Tµ/)½ÃÈd[Ü(E¨ëè³ eÉÍaJÀ¬5.pp¹îß-YáfAßíjñV;^@ìp;WYÙ+0¢GÑÔeô°0Ç·[cvÙU±DÇð5£|? éòÜq7ó5U¤¾§Áy:£81æ>Ý®ñÿV0F53a¯ÀZwT®W?¹L
